In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import eclipse_calc

from harc_plot import geopack
%matplotlib inline

Re = 6371 # Radius of the Earth in km

In [2]:
class Obscuration(object):
    def __init__(self,date_str='20170821',data_dir='data/eclipse_calc/'):
        csv_path = os.path.join(data_dir,date_str)

        fpaths = glob.glob(os.path.join(csv_path,'*.csv.bz2'))
        fpaths.sort()

        # Load Eclipses
        df    = None
        dates = []
        files = []
        for fpath in fpaths:
            bname = os.path.basename(fpath)
            files.append(bname)

            date  = datetime.datetime.strptime(bname[:13],'%Y%m%d.%H%M')
            dates.append(date)
            cname = 'obsc-{!s}'.format(date.strftime('%H%M'))
            alt   = str(bname[14:17])

            dft   = pd.read_csv(fpath,comment='#')
            dft.rename(columns={'obsc':cname},inplace=True)
            if df is None:
                df = dft.copy()
            else:
                df[cname] = dft[cname]
        
        # Calculate max obscuration in each cell.
        tf = df.columns.str.contains('obsc-*')
        obsc_df = df.loc[:,tf]

        # Store max obscuration in new dataframe.
        max_obsc = df.loc[:,~tf].copy()
        max_obsc['max_obsc'] = obsc_df.max(1)
        
        # Store eclipse data on object.
        self.files    = files
        self.dates    = dates
        self.obsc_df  = df
        self.max_obsc = max_obsc
        
    def plot_map(self):
        max_obsc = self.max_obsc
        
        fig = plt.figure(figsize=(20,16))
        crs = ccrs.PlateCarree()
        ax  = fig.add_subplot(111,projection=crs)
        sTime = min(self.dates)
        eTime = max(self.dates)
        hmap = eclipse_calc.maps.HamMap(sTime,eTime,ax=ax,crs=crs)
        hmap.overlay_gridsquares(label_precision=0,major_style={'color':'0.8','linestyle':'--'})
        hmap.overlay_gridsquare_data(max_obsc['grid'],max_obsc['max_obsc'],cmap=mpl.cm.gray_r,cbar_shrink=0.5,
                                     vmin=0,vmax=1,zorder=5,cbar_label='Obscuration')
        return (fig, ax)

In [4]:
date_keys = []
# date_keys.append('20170821')
# date_keys.append('20231014')
date_keys.append('20240408')

for date_key in date_keys:
    obsc_obj = Obscuration(date_key)
    fig, ax = obsc_obj.plot_map()
    
    # US Limits
    ax.set_xlim(-130,-60)
    ax.set_ylim(20,55)
    
    plt.show()

OSError: Invalid data stream